# Benchmarking: Julia

In [1]:
using Random

In [2]:
rng = Random.seed!(0)

TaskLocalRNG()

## Objective Functions

We will be using the [Levy3](http://infinity77.net/global_optimization/test_functions_nd_L.html#go_benchmark.Levy03), [Michalewicz](https://www.sfu.ca/~ssurjano/michal.html), and [Ackley](https://www.sfu.ca/~ssurjano/ackley.html) objective functions for benchmarking.

In [3]:
function levy3(x)
    n = length(x)

    y(x_i) = 1 + (x_i-1)/4

    term1 = sin(π*y(x[1]))^2
    term3 = y(x[n]-1)^2 * (1+sin(2π*y(x[n]))^2)
    
    sum = 0
    for i=1:n-1
        new = (y(x[i])-1)^2*(1+10sin(π*y(x[i])+1)^2)
        sum += new
    end

    return term1 + sum + term3
end

function michalewicz(x; m=10)
    sleep(0.01)
    return -sum(sin(v)*sin(i*v^2/π)^(2m) for
               (i,v) in enumerate(x))
end

function ackley(x; a=20, b=0.2, c=2π)
    sleep(0.01)
    d = length(x)
    return -a*exp(-b*sqrt(sum(x.^2)/d)) -
              exp(sum(cos.(c*xi) for xi in x)/d) + a + exp(1)
end

ackley (generic function with 1 method)

## Genetic Algorithm (GA) Implementation

The following section defines a GA implementation using a real-valued representation. The Selection, Crossover, and Mutation types will be reused in both the baseline and parallelized versions of the algorithm.

### Population

In [4]:
function rand_population_uniform(m, a, b)
    d = length(a)
    return [a+rand(d).*(b-a) for i in 1:m]
end

rand_population_uniform (generic function with 1 method)

### Selection

In [5]:
abstract type SelectionMethod end
struct TruncationSelection <: SelectionMethod
    k # top k to keep
end
function select(t::TruncationSelection, y)
    p = sortperm(y)
    return [p[rand(1:t.k, 2)] for i in y]
end

struct TournamentSelection <: SelectionMethod
    k
end
function select(t::TournamentSelection, y)
    getparent() = begin
        p = randperm(length(y))
        p[argmin(y[p[1:t.k]])]
    end
    return [[getparent(), getparent()] for i in y] 
end

struct RouletteWheelSelection <: SelectionMethod end
function select(::RouletteWheelSelection, y)
    y = maximum(y) .- y
    cat = Categorical(normalize(y, 1))
    return [rand(cat, 2) for i in y]
end

select (generic function with 3 methods)

### Crossover

In [6]:
abstract type CrossoverMethod end
struct SinglePointCrossover <: CrossoverMethod end
function crossover(::SinglePointCrossover, a, b)
    i = rand(1:length(a))
    return vcat(a[1:i], b[i+1:end])
end

struct TwoPointCrossover <: CrossoverMethod end
function crossover(::TwoPointCrossover, a, b)
    n = length(a)
    i, j = rand(1:n, 2)
    if i > j
        (i,j) = (j,i)
    end
    return vcat(a[1:i], b[i+1:j], a[j+1:n])
end

struct UniformCrossover <: CrossoverMethod end
function crossover(::UniformCrossover, a, b)
    child = copy(a)
    for i in 1 : length(a)
        if rand() < 0.5
            child[i] = b[i]
        end
    end
    return child
end

crossover (generic function with 3 methods)

### Mutation

In [7]:
abstract type MutationMethod end
struct BitwiseMutation <: MutationMethod
    λ
end
function mutate(M::BitwiseMutation, child)
    return [rand() < M.λ ? !v : v for v in child]
end

struct GaussianMutation <: MutationMethod
    σ
end
function mutate(M::GaussianMutation, child)
    return child + randn(length(child))*M.σ
end

mutate (generic function with 2 methods)

### Algorithm

First, ensure that we have multithreading enabled. The easiest way to do this is to specify the number of threads when starting up the notebook:

`JULIA_NUM_THREADS=4 jupyter-notebook`

In [8]:
Threads.nthreads() # should be > 1

10

In [9]:
function genetic_algorithm(f, population, k_max, S, C, M; parallel=false)
    n = length(population)
    for k in 1 : k_max
        if parallel
            # Perform function evaluations in parallel
            f_pop = zeros(n)
            Threads.@threads for i=1:n
               f_pop[i] = f(population[i]) 
            end
        else
            f_pop = f.(population)
        end
        parents = select(S, f_pop)
        children = [crossover(C, population[p[1]], population[p[2]]) for p in parents]
        population .= mutate.(Ref(M), children)
    end
    population[argmin(f.(population))]
end

genetic_algorithm (generic function with 1 method)

### Benchmarking (baseline)

In [10]:
f = x -> michalewicz(x)

S = TruncationSelection(10)
C = SinglePointCrossover()
M = GaussianMutation(0.1)

k_max = 10
m = 40
population = rand_population_uniform(m, [0.0, 0.0], [4.0, 4.0])

40-element Vector{Vector{Float64}}:
 [1.6227978835681167, 0.2741832975460601]
 [3.4485634287819398, 0.3438834634336878]
 [2.646450762923295, 0.46530941532634396]
 [0.4375424085791564, 2.8080177767349186]
 [1.1580393692877515, 0.11419991066393598]
 [2.154557655862612, 3.5879591610268338]
 [1.0339112614534827, 1.3557962068886953]
 [1.6989666560852252, 3.6189103070386164]
 [2.978718331949238, 0.6222297668781192]
 [2.3019615369020383, 1.816925890183565]
 [0.0668971086636172, 1.9831175619969552]
 [2.542710434871043, 0.34831495149735403]
 [1.7036205409667047, 2.2944474777203316]
 ⋮
 [3.4836989406410375, 3.0308454801035754]
 [0.03292787614795989, 2.426377650783829]
 [0.46407739230089184, 0.6806902469822944]
 [3.172635826731833, 0.8710915371037222]
 [3.6521526529050505, 1.6390326906675292]
 [2.4975358217191, 0.22630958404098145]
 [3.487549644688445, 1.4215005383254011]
 [2.09603285944997, 1.119627276969072]
 [0.8611716773442128, 1.5182730273241658]
 [2.308724747676947, 1.1271127292587386]
 [3.

In [11]:
@time genetic_algorithm(f, population, k_max, S, C, M)

  6.231762 seconds (2.48 M allocations: 137.127 MiB, 0.78% gc time, 13.83% compilation time)


2-element Vector{Float64}:
 2.2204334208424057
 1.5400543380262424

In [12]:
@time genetic_algorithm(f, population, k_max, S, C, M; parallel=true)

  1.579789 seconds (61.82 k allocations: 3.538 MiB, 0.82% gc time, 10.47% compilation time)


2-element Vector{Float64}:
 2.220092878471803
 1.5655794177367475